## Create a Customer Travel Destinations Map.

In [37]:
# Import the dependencies
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the weatherpy_database.csv file. 
file_to_load = "../weather_database/weatherpy_database.csv"
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,39.22,81,40,18.41,scattered clouds
1,1,Butaritari,KI,3.0707,172.7902,82.02,77,12,15.64,light rain
2,2,Bambous Virieux,MU,-20.3428,57.7575,73.65,73,40,3.44,scattered clouds
3,3,Huarmey,PE,-10.0681,-78.1522,65.97,69,25,5.84,scattered clouds
4,4,Yarada,IN,17.6500,83.2667,86.32,79,40,4.61,scattered clouds


In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.0707,172.7902,82.02,77,12,15.64,light rain
4,4,Yarada,IN,17.6500,83.2667,86.32,79,40,4.61,scattered clouds
5,5,Ponta Do Sol,PT,32.6667,-17.1000,78.01,69,40,2.17,scattered clouds
8,8,Atuona,PF,-9.8000,-139.0333,77.92,68,2,20.24,clear sky
9,9,Novosergiyevka,RU,52.0934,53.6528,76.53,62,97,7.61,overcast clouds
14,14,Hatillo,PR,18.4863,-66.8254,89.20,74,97,8.01,overcast clouds
18,18,Norsup,VU,-16.0667,167.3833,79.23,85,96,11.45,overcast clouds
19,19,Praia,CV,14.9215,-23.5087,86.54,42,20,14.97,few clouds
26,26,Albany,US,42.6001,-73.9662,81.28,52,4,12.28,clear sky
34,34,Boone,US,39.0334,-92.3335,83.39,62,100,2.80,overcast clouds


In [35]:
# Check for null values
# preferred_cities_df.count()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# Create DataFrame to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# Create a new column 
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,82.02,light rain,3.0707,172.7902,
4,Yarada,IN,86.32,scattered clouds,17.6500,83.2667,
5,Ponta Do Sol,PT,78.01,scattered clouds,32.6667,-17.1000,
8,Atuona,PF,77.92,clear sky,-9.8000,-139.0333,
9,Novosergiyevka,RU,76.53,overcast clouds,52.0934,53.6528,
14,Hatillo,PR,89.20,overcast clouds,18.4863,-66.8254,
18,Norsup,VU,79.23,overcast clouds,-16.0667,167.3833,
19,Praia,CV,86.54,few clouds,14.9215,-23.5087,
26,Albany,US,81.28,clear sky,42.6001,-73.9662,
34,Boone,US,83.39,overcast clouds,39.0334,-92.3335,


In [17]:
# Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # Create base url for the API request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [45]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,82.02,light rain,3.0707,172.7902,Isles Sunset Lodge
4,Yarada,IN,86.32,scattered clouds,17.6500,83.2667,Yarada Jungle Beach Resorts
5,Ponta Do Sol,PT,78.01,scattered clouds,32.6667,-17.1000,Hotel do Campo
8,Atuona,PF,77.92,clear sky,-9.8000,-139.0333,Villa Enata
9,Novosergiyevka,RU,76.53,overcast clouds,52.0934,53.6528,Gostinichnyy Kompleks Edem


In [54]:
# Export the hotel data into a CSV
output_data_file = "weatherpy_vacation.csv"
clean_hotel_df.to_csv(output_data_file, index=False)

In [50]:
# Create the info box template for the map
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [53]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display the figure
fig


Figure(layout=FigureLayout(height='420px'))